In [27]:
%matplotlib inline
import pandas as pd
import numpy as np
import plotly 
import plotly.plotly as py 
import plotly.figure_factory as ff
from plotly.graph_objs import *
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import scipy.stats as stats
import math
from sklearn import linear_model
from sklearn.linear_model import SGDRegressor

In [28]:
monthlyAsylumSeekers = pd.read_csv("../data/refugee_monthly.csv", sep=",", engine='python', encoding ='latin1')   
monthlyAsylumSeekers = monthlyAsylumSeekers[monthlyAsylumSeekers['Value'] != '*']
monthlyAsylumSeekers['Value'] = monthlyAsylumSeekers['Value'].apply(pd.to_numeric)
monthlyAsylumSeekers.head()

,Country / territory of asylum/residence,Origin,Year,Month,Value
0,Greece,Georgia,2008,March,140
1,Greece,Georgia,2008,April,199
2,Greece,Georgia,2008,May,210
3,Greece,Georgia,2008,June,208
4,Greece,Georgia,2008,July,224


In [29]:
monthlyAsylumSeekers.sort_values(by=['Year'])
monthlyAsylumSeekers.rename(columns={'Country / territory of asylum/residence':'Country_territoryofasylum_residence'}, inplace=True)
monthlyAsylumSeekers.head()

,Country_territoryofasylum_residence,Origin,Year,Month,Value
0,Greece,Georgia,2008,March,140
1,Greece,Georgia,2008,April,199
2,Greece,Georgia,2008,May,210
3,Greece,Georgia,2008,June,208
4,Greece,Georgia,2008,July,224


In [30]:
from sklearn import preprocessing
countryTrainer=preprocessing.LabelEncoder()
originTrainer=preprocessing.LabelEncoder()
monthTrainer=preprocessing.LabelEncoder()

Transform categorical into numerical

In [31]:

monthlyAsylumSeekers['Country_territoryofasylum_residence']=countryTrainer.fit_transform(monthlyAsylumSeekers['Country_territoryofasylum_residence'])
monthlyAsylumSeekers['Month']=monthTrainer.fit_transform(monthlyAsylumSeekers['Month'])
monthlyAsylumSeekers['Origin']=originTrainer.fit_transform(monthlyAsylumSeekers['Origin'])
monthlyAsylumSeekers[['Country_territoryofasylum_residence','Origin', 'Month']].head()

,Country_territoryofasylum_residence,Origin,Month
0,15,68,7
1,15,68,0
2,15,68,8
3,15,68,6
4,15,68,5


Test to see if it works

In [32]:
print monthTrainer.inverse_transform([0,1,2,3,4,5,6,7,8,9,10,11])
print monthTrainer.transform(['March','April','May'])
print countryTrainer.transform(['Greece'])
print originTrainer.transform(['Georgia'])

[u'April' u'August' u'December' u'February' u'January' u'July' u'June'
 u'March' u'May' u'November' u'October' u'September']
[7 0 8]
[15]
[68]


Gonna try SGD classifier first so y needs to be categorical country of asylum

In [33]:
X=monthlyAsylumSeekers[['Value','Origin','Month']]
y=monthlyAsylumSeekers['Country_territoryofasylum_residence']
y.head()

0    15
1    15
2    15
3    15
4    15
Name: Country_territoryofasylum_residence, dtype: int64

Test linear regression (not important, but wanted to see)

In [34]:
lm = linear_model.LinearRegression()
# create a fitted model for the countries
#lm = smf.ols(formula='Country_territoryofasylum_residence ~ Value + Origin + Month', data=monthlyAsylumSeekers).fit()
#lm=smf.ols(Y, X).fit()
model=lm.fit(X,y)

In [35]:
prediction=model.predict([[200,originTrainer.transform(['Afghanistan'])[0],monthTrainer.transform(['March'])[0]]]).tolist()
print prediction

[26.589428543664752]


SGDClassifier to predict where a country of asylum based on value, month, and country of origin

In [62]:
clf = linear_model.SGDClassifier(loss="hinge", penalty="l2")
clf.fit(X, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

Test

In [63]:
clf.predict([[100,originTrainer.transform(['Armenia'])[0],monthTrainer.transform(['April'])[0]]])

array([43], dtype=int64)

In [64]:
print countryTrainer.inverse_transform(43)

USA (INS/DHS)
